In [ ]:
# Importing the packages we will need
from __future__ import print_function
from __future__ import division
from IPython.core.display import HTML
from IPython.display import display, clear_output
from ipywidgets import Button, Layout, GridBox, ButtonStyle, HBox, VBox, Label, Dropdown
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
from matplotlib import pyplot as plt

import ipywidgets as ipw
%matplotlib inline
import pandas as pd
import math
import numpy as np
import xlrd
import xlsxwriter
import os.path
import glob
import warnings
import platform
import time

import scipy
from scipy.io import loadmat
from scipy.sparse import linalg
import os
import sys

HTML("""
<style>
#notebook-container{
    box-shadow: none !important;
}

.body {
    max-width: 960px !important;
}

.container {
    max-width: 960px !important;
}

.notebook_app {
    background: #fff !important;
}

.navbar-default {
    background: none;
    border: none;
}

.navbar-default .navbar-nav > li > a:hover, #kernel_indicator:hover {
    border-bottom: 2px solid #fff;
    color: rgba(255, 255, 255, 1);
}

div.input_area {
    border: none;
    border-radius: 0;
    background: #f7f7f7;
    line-height: 1.5em;
    margin: 0.5em 0;
    padding: 0;
    max-width: 960px !important;
}

div.cell {
    transition: all 0.25s;
    border: none;
    position: relative;
    top: 0;
    max-width: 960px !important;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border: none;
    background: transparent;
    box-shadow: 0 6px 18px #aaa;
    z-index: 10;
    top: -10px;
    max-width: 960px !important;
}


div#pager {
    opacity: 0.85;
    z-index: 9999;
    max-width: 960px !important;
}

.navbar-default .navbar-nav > .open > a, .navbar-default .navbar-nav > .open > a:hover, .navbar-default .navbar-nav > .open > a:focus {
    color: #fff;
    background-color: transparent;
    border-bottom: 2px solid #fff;
}

.dropdown-menu {
    z-index: 999999 !important;
    opacity: 0.95;
    max-width: 960px !important;
}

.dropdown-menu > li > a {
    color: #fff;
}

.dropdown-menu > .disabled > a, .dropdown-menu > .disabled > a:hover, .dropdown-menu > .disabled > a:focus {
    color: rgba(255, 255, 255, 0.25);
}

.navbar-nav > li > .dropdown-menu {
    border: none;
    box-shadow: none;
}

div.output_wrapper {
    background: #fff;
}

div.cell.selected .div.output_scroll {
    box-shadow: none;
    max-width: 960px !important;
}

div.output_wrapper {
    margin: 0 0 1em;
    transition: all 0.25s;
    max-width: 960px !important;
}

div.cell.selected .output_wrapper {
    margin: 0;
}

.dataframe {
    background: #fff;
    box-shadow: 0px 1px 2px #bbb;
}

.dataframe thead th, .dataframe tbody td {
    text-align: right;
    padding: 1em;
}

.output, div.output_scroll {
    box-shadow: none;
}

.rendered_html pre code {
    background: #f4f4f4;
    border: 1px solid #ddd;
    border-left: 3px solid #2a7bbd;
    color: #444;
    page-break-inside: avoid;
    font-family: monospace;
    font-size: 15px;
    line-height: 1.6;
    margin-bottom: 1.6em;
    max-width: 100%;
    overflow: auto;
    padding: 1em 1.5em;
    display: block;
    word-wrap: break-word;
}

h1, .h1 {
    font-size: 33px;
    font-family: "Trebuchet MS";
    font-size: 2.5em !important;
    color: #2a7bbd;
}
</style>
""")

In [ ]:
def getvalue(x): 
    return x

# Make user interface

style = {'description_width': '170px'}
w1 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=3.0,
    min=0,
    max=30.0,
    step=0.1,
    description='$a_0$: Max accu in m year$^{-1}$:',
    style=style,
    disabled=False
))
# display(w1)

w2 = interactive(getvalue, x=ipw.BoundedFloatText(
    value=0.01,
    min=0,
    max=0.01,
    step=0.0001,
    description='$a_1$: Accu slope in year$^{-1}$:',
    style=style,
    disabled=False
))
# display(w2)

In [ ]:
def ice_flow_model(accu, accu_slope): 
    A_0 = 1e-24
    rho_i = 910
    g = 9.81
    n = 3
    C = 2.0* A_0 * (rho_i*g)**n/(n+1)

    x=np.linspace(0, 1000, num=50)
    bed=-x/100
    h=100 - np.power(x/100,2)
    i_0 = np.where( h< 0)
    h[i_0]=0
    h=h.astype('float64')
    s=bed+h
    accu_array=(accu-x*accu_slope)/(365*24*60*60)

    # Data for plotting
    t = np.linspace(0, 1000*(365*24*60*60), num=10000*len(x))
    L=0*t

    delta_t=t[2]-t[1]
    delta_x=x[2]-x[1]
    k=0

    fig, ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5, forward=True)
    for i in t/(365*24*60*60):
        s=bed+h
        ds_dx_p=(s[2:]-s[1:-1])/delta_x   # 1.5 to end-0.5
        ds_dx_m=(s[1:-1]-s[:-2])/delta_x  # 0.5 to end-1.5

        h_np1=np.power(h,(n+1))

        q_p=-(h_np1[2:]   + h_np1[1:-1])/2.0 * np.power((np.abs(ds_dx_p)),(n-1.0))*ds_dx_p
        q_m=-(h_np1[1:-1] + h_np1[:-2] )/2.0 * np.power((np.abs(ds_dx_m)),(n-1.0))*ds_dx_m

        dq_dx=q_p-q_m  # 1.0 to end-1.0

        q=0*h
        q[1:-1] = C*(q_p+q_m)/2.0
        u=q/(np.abs(h)+1e-6)

        h[1:-1]=h[1:-1] - C*delta_t/delta_x*dq_dx + accu_array[1:-1]*delta_t

        h[0] = h[1] + bed[1] - bed[0]
        h[-1] = h[-2] + bed[-2] - bed[-1]
        i_0 = np.where( h < 0)
        h=np.where(h <= 0, 0, h)

        u=np.where(h < 0, 0, u)
        u[:-1]=np.where(h[1:] <= 0, 0, u[:-1])
        i_0=np.nonzero(h[:-1]) 
        #u[i_0]=0
        #u[i_0[0]-1]=0
        L[k]=x[np.argmax(i_0)]
        h[np.argmax(i_0)+1:]=0 # just in case, setting to zero what should be zero

        if k % (1000*len(x))==0:

            plt.clf()  

            ax1 = plt.subplot(2, 2, 1)
            ax1.plot(x/1000, accu_array*(365*24*60*60))
            ax1.set(xlabel='x [km]', ylabel='accu [m/yr]',
                   title='About as simple as it gets, t=%i years' %round(i,0))
            ax1.grid()

            ax2 = plt.subplot(2, 2, 2)

            ax2.plot(x/1000, bed)
            ax2.plot(x/1000, bed+h)

            ax2.set(xlabel='x [km]', ylabel='z [m]')
            ax2.grid()

            ax3 = plt.subplot(2, 2, 3)
            ax3.plot(x/1000, u*(365*24*60*60))
            ax3.set(xlabel='x [km]', ylabel='u [m/yr]')

            ax4 = plt.subplot(2, 2, 4)
            ax4.plot(t[:k-1]/(365*24*60*60), L[:k-1]/1000)
            ax4.set(xlabel='t [yrs]', ylabel='L [km]', xlim=(0, 1000))  

            display(plt.gcf())
            
            clear_output(wait=True)

        k=k+1

    plt.clf()  
    ax1 = plt.subplot(2, 2, 1)
    ax1.plot(x/1000, accu_array*(365*24*60*60))

    ax1.set(xlabel='x [km]', ylabel='accu [m/yr]',
           title='About as simple as it gets, t=%i years' %round(i,0))
    ax1.grid()

    ax2 = plt.subplot(2, 2, 2)
    ax2.plot(x/1000, bed)
    ax2.plot(x/1000, bed+h)

    ax2.set(xlabel='x [km]', ylabel='z [m]')
    ax2.grid()

    ax3 = plt.subplot(2, 2, 3)
    ax3.plot(x/1000, u*(365*24*60*60))
    ax3.set(xlabel='x [km]', ylabel='u [m/yr]')

    ax4 = plt.subplot(2, 2, 4)
    ax4.plot(t[:k-1]/(365*24*60*60), L[:k-1]/1000)
    ax4.set(xlabel='t [yrs]', ylabel='L [km]')

    display(plt.gcf())
    clear_output(wait=True)

# KE3001 Mass balance practical
Welcome! In this practical you will run a real ice sheet model that solves for ice sheet mass balance and velocity. 

## What you will learn
<ul>
<li> Principles of ice sheet and glacier mass balance
<li> Creating plots in Excel, incl. fitting data with a trendline.
</ul>

## Part 1: Ice sheet mass balance
In the model below, the surface mass balance (accumulation and ablation) is given by $$\dot a=(\dot a_0 - a_1 \times x).$$
<ol>
<li> Run the model into steady state for 5 different values of $a_0$ between 1 m year$^{-1}$ and 5.0 m year$^{-1}$. Note the values of $a_0$, the value of $a_1$, and the final length of the glacier for each of your runs.
<li> Run the model into steady state for 5 different values of $a_1$ between 0.1 year$^{-1}$ and 1.0 year$^{-1}$. Note the value of $a_0$, the values of $a_1$, and the final length of the glacier for each of your runs.
<li> Create a scatter graph with $a_0$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Add a lienar trendline and note the equation of the trendline.
<li> Create a scatter graph with $a_1$ on the $x$-axis and the final glacier length on the $y$ axis. What do you observe? Add a linear trendline and note the equation of the trendline. 
<li> Would there have been a faster way to determine these relationships?
</ol>

In [ ]:
#left_box  = ipw.VBox([w1, w2, w3, w4])
#right_box = ipw.VBox([w2, w6, w7, w8])
#ipw.HBox([left_box,right_box])

left_box  = ipw.VBox([w1])
right_box = ipw.VBox([w2])
ipw.HBox([left_box,right_box])

In [ ]:

# Loading the input parameters defined by the user
button1 = ipw.Button(description="Run the model!")
output1 = ipw.Output()

display(button1, output1)

global iter
iter=0
def on_button_clicked(b):
    with output1:
        clear_output(wait=True)
        accu=w1.result
        accu_slope=w2.result
        
        global iter
        iter=iter+1 
        
        ice_flow_model(accu, accu_slope)
button1.on_click(on_button_clicked)

## Part 2: Mass balance-elevation feedback
We are now running a modified version of the from part 1. Now the surface mass balance (accumulation and ablation) is given by $$\dot a=(\dot a_0 - a_1 \times x + a_2\times s).$$

$s$ is the surface height of the glacier, and $a_2$ is the lapse rate.

<ol>
<li> Starting from the default values of the glacier, run the model into steady state for 5 different values of $a_0$ between 1 m year$^{-1}$ and 0.0 m year$^{-1}$, decreasing the value for each run. Note the values of $a_0$, the value of $a_1$, and the final length of the glacier for each of your runs.
</ol>